In [1]:
from korea_news_crawler.articlecrawler import ArticleCrawler

In [3]:
Crawler = ArticleCrawler()  
Crawler.set_category("정치", "IT과학", "economy")  
Crawler.set_date_range(2020, 1, 2020, 4)  
Crawler.start()

{'start_year': 2020, 'start_month': 1, 'end_year': 2020, 'end_month': 4}


In [8]:
# 가져올 범위를 정의
# 2015-02-25 ~ 2015-02-28 // 2015-03-01 ~ 2015-03-30

import datetime

days_range = []

start = datetime.datetime.strptime("2019-01-01", "%Y-%m-%d")
end = datetime.datetime.strptime("2019-01-31", "%Y-%m-%d") # 범위 + 1
date_generated = [start + datetime.timedelta(days=x) for x in range(0, (end-start).days)]

for date in date_generated:
    days_range.append(date.strftime("%Y-%m-%d"))

print(days_range)

['2019-01-01', '2019-01-02', '2019-01-03', '2019-01-04', '2019-01-05', '2019-01-06', '2019-01-07', '2019-01-08', '2019-01-09', '2019-01-10', '2019-01-11', '2019-01-12', '2019-01-13', '2019-01-14', '2019-01-15', '2019-01-16', '2019-01-17', '2019-01-18', '2019-01-19', '2019-01-20', '2019-01-21', '2019-01-22', '2019-01-23', '2019-01-24', '2019-01-25', '2019-01-26', '2019-01-27', '2019-01-28', '2019-01-29', '2019-01-30']


In [9]:
import requests
from bs4 import BeautifulSoup

def get_bs_obj(url):
    result = requests.get(url)
    bs_obj = BeautifulSoup(result.content, "html.parser")
    
    return bs_obj

In [10]:
from tqdm import tqdm_notebook

test = ["2015-03-01"] # 테스트를 위한 데이터 수집 구간

for date in tqdm_notebook(test):
    
    news_arrange_url = "https://news.naver.com/main/history/mainnews/list.nhn"
    news_list_date_page_url = news_arrange_url + "?date=" + date

    # get bs_obj
    bs_obj = get_bs_obj(news_list_date_page_url)
    
    # 포토 뉴스 페이지 수 구하기
    photo_news_count = bs_obj.find("div", {"class": "eh_page"}).text.split('/')[1]
    photo_news_count = int(photo_news_count)
    
    print(photo_news_count)
    
    # 리스트 뉴스 페이지 수 구하기
    text_news_count = bs_obj.find("div", {"class": "mtype_list_wide"}).find("div", {"class": "eh_page"}).text.split('/')[1]
    text_news_count = int(text_news_count)
    
    print(text_news_count)

AttributeError: 'NoneType' object has no attribute 'text'

In [11]:
# 가져올 범위를 정의
# 2015-02-25 ~ 2015-02-28 // 2015-03-01 ~ 2015-03-31

import datetime

days_range = []

start = datetime.datetime.strptime("2015-02-25", "%Y-%m-%d") # 수집 시작 날짜
end = datetime.datetime.strptime("2015-03-31", "%Y-%m-%d") # 수집 종료 날짜 + 1
date_generated = [start + datetime.timedelta(days=x) for x in range(0, (end-start).days)]

for date in date_generated:
    days_range.append(date.strftime("%Y-%m-%d"))

# 크롤러 작성

import requests
import pandas as pd
from bs4 import BeautifulSoup
from tqdm import tqdm_notebook

main_news_list = []

# html parser 정의
def get_bs_obj(url):
    result = requests.get(url)
    bs_obj = BeautifulSoup(result.content, "html.parser")
    
    return bs_obj
    
for date in tqdm_notebook(days_range):
    
    news_arrange_url = "https://news.naver.com/main/history/mainnews/list.nhn"
    news_list_date_page_url = news_arrange_url + "?date=" + date

    # get bs_obj
    bs_obj = get_bs_obj(news_list_date_page_url)
    
    # 포토 뉴스 페이지 수 구하기
    photo_news_count = bs_obj.find("div", {"class": "eh_page"}).text.split('/')[1]
    photo_news_count = int(photo_news_count)
    
    # 리스트 뉴스 페이지 수 구하기
    text_news_count = bs_obj.find("div", {"class": "mtype_list_wide"}).find("div", {"class": "eh_page"}).text.split('/')[1]
    text_news_count = int(text_news_count)
    
    # 포토 뉴스 부분 링크 크롤링
    for page in tqdm_notebook(range(1,photo_news_count+1)):
        
        # 포토 뉴스 링크
        news_list_photo_url = 'http://news.naver.com/main/history/mainnews/photoTv.nhn'
        date_str = "?date="
        page_str = "&page="
        news_list_photo_full_url = news_list_photo_url + "?date=" + date + "&page=" + str(page)
        
        # get bs obj
        photo_bs_obj = get_bs_obj(news_list_photo_full_url)
        
        # 링크 내 정보 수집
        ul = photo_bs_obj.find("ul", {"class": "edit_history_lst"})
        lis = ul.find_all("li")
        for item in lis:
            title = item.find("a")["title"]
            press = item.find("span", {"class" : "eh_by"}).text
            
            # link
            link = item.find("a")["href"]
            
            sid1 = link.split('&')[-3].split('=')[1]
            oid = link.split('&')[-2].split('=')[1]
            aid = link.split('&')[-1].split('=')[1]            
            
            # 연예 TV 기사 제외
            if sid1 == "shm":
                continue
                
            article_type = "pic"
            
            pic_list = [date, article_type, title, press, sid1, link, aid]
            
            main_news_list.append(pic_list)
 
    # 텍스트 뉴스 부분 링크 크롤링
    for page in tqdm_notebook(range(1, text_news_count+1)):
        
        # 텍스트 뉴스 링크
        news_list_text_url = 'http://news.naver.com/main/history/mainnews/text.nhn'
        date_str = "?date="
        page_str = "&page="
        news_list_text_full_url = news_list_text_url + "?date=" + date + "&page=" + str(page)
        
        # get bs obj
        text_bs_obj = get_bs_obj(news_list_text_full_url)
        
        # 링크 내 정보 수집
        uls = text_bs_obj.find_all("ul")
        for ul in uls:
            lis = ul.find_all("li")
            for item in lis:
                title = item.find("a").text
                press = item.find("span", {"class" : "writing"}).text
                
                # link
                link = item.find("a")["href"]

                sid1 = link.split('&')[-3].split('=')[1]
                oid = link.split('&')[-2].split('=')[1]
                aid = link.split('&')[-1].split('=')[1]
                
                # 연예 TV 기사 제외
                if sid1 == "shm":
                    continue

                article_type = "text"
                
                text_list = [date, article_type, title, press, sid1, link, aid]
                
                main_news_list.append(text_list)


# make .csv file
naver_news_main_df = pd.DataFrame(main_news_list, 
                                  columns = ["date", "type", "title", "press", "category", "link", "aid"])
naver_news_main_df.to_csv("naver_main_news_{}_to_{}.csv".format(days_range[0], days_range[-1]), index=False)

print("=== total # of articles is {} ===".format(len(main_news_list)))

AttributeError: 'NoneType' object has no attribute 'text'

In [12]:
from tqdm import tqdm_notebook
from pprint import pprint

test = ["2015-03-01"]
main_news_list = []

for date in tqdm_notebook(test):
    
    news_arrange_url = "https://news.naver.com/main/history/mainnews/list.nhn"
    news_list_date_page_url = news_arrange_url + "?date=" + date

    # get bs_obj
    bs_obj = get_bs_obj(news_list_date_page_url)
    
    # 포토 뉴스 페이지 수 구하기
    photo_news_count = bs_obj.find("div", {"class": "eh_page"}).text.split('/')[1]
    photo_news_count = int(photo_news_count)
    
    # 리스트 뉴스 페이지 수 구하기
    text_news_count = bs_obj.find("div", {"class": "mtype_list_wide"}).find("div", {"class": "eh_page"}).text.split('/')[1]
    text_news_count = int(text_news_count)
    
    # 포토 뉴스 부분 링크 크롤링
    for page in tqdm_notebook(range(1,photo_news_count+1)):
        
        # 포토 뉴스 링크
        news_list_photo_url = 'http://news.naver.com/main/history/mainnews/photoTv.nhn'
        date_str = "?date="
        page_str = "&page="
        news_list_photo_full_url = news_list_photo_url + "?date=" + date + "&page=" + str(page)
        
        # get bs obj
        photo_bs_obj = get_bs_obj(news_list_photo_full_url)
        
        # 링크 내 정보 수집
        ul = photo_bs_obj.find("ul", {"class": "edit_history_lst"})
        lis = ul.find_all("li")
        for item in lis:
            title = item.find("a")["title"]
            press = item.find("span", {"class" : "eh_by"}).text
            
            # link
            link = item.find("a")["href"]
            
            sid1 = link.split('&')[-3].split('=')[1]
            oid = link.split('&')[-2].split('=')[1]
            aid = link.split('&')[-1].split('=')[1]            
            
            # 연예 TV 기사 제외
            if sid1 == "shm":
                continue
                
            article_type = "pic"
            
            pic_list = [date, article_type, title, press, sid1, link, aid]
            
            main_news_list.append(pic_list)
            
    # 텍스트 뉴스 부분 링크 크롤링
    for page in tqdm_notebook(range(1, text_news_count+1)):
        
        # 텍스트 뉴스 링크
        news_list_text_url = 'http://news.naver.com/main/history/mainnews/text.nhn'
        date_str = "?date="
        page_str = "&page="
        news_list_text_full_url = news_list_text_url + "?date=" + date + "&page=" + str(page)
        
        # get bs obj
        text_bs_obj = get_bs_obj(news_list_text_full_url)
        
        # 링크 내 정보 수집
        uls = text_bs_obj.find_all("ul")
        for ul in uls:
            lis = ul.find_all("li")
            for item in lis:
                title = item.find("a").text
                press = item.find("span", {"class" : "writing"}).text
                
                # link
                link = item.find("a")["href"]

                sid1 = link.split('&')[-3].split('=')[1]
                oid = link.split('&')[-2].split('=')[1]
                aid = link.split('&')[-1].split('=')[1]
                
                # 연예 TV 기사 제외
                if sid1 == "shm":
                    continue

                article_type = "text"
                
                text_list = [date, article_type, title, press, sid1, link, aid]
                
                main_news_list.append(text_list)

pprint(main_news_list, width = 20)

AttributeError: 'NoneType' object has no attribute 'text'